In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
import numpy as np
from keras import metrics
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [21]:
def import_data(path, sep): 
    balance_data = pd.read_csv( path, 
    sep= sep, header = None) 
      
#     Printing the dataset shape 
#     print ("Dataset Lenght: ", len(balance_data)) 
#     print ("Dataset Shape: ", balance_data.shape) 
#     Printing the dataset obseravtions 
#     print ("Dataset: ",balance_data.head()) 
    return balance_data 

In [22]:
def split_dataset(data):
#     distributed data into input parameters X and predicitons Y
#     data = np.asarray(data)
    data = data.loc[1:, :]
    X = data.values[:, :-3]
    Y = data.values[:, -3:]

#     splitting into training and testing datasets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 42)
    
    X_train = np.asarray(X_train, dtype=np.float32)
    Y_train = np.asarray(Y_train, dtype=np.float32)
    X_test = np.asarray(X_test, dtype=np.float32)
    Y_test = np.asarray(Y_test, dtype=np.float32)
        
    return X_train, Y_train, X_test, Y_test

In [23]:
path = "./not16nm_Delays_process_temp_pvdd_cqloadNEWLOAD.csv"
path1 = "./AND2_16nm_stat00.csv"
data = import_data(path, ',')
train_x, train_y, test_x, test_y = split_dataset(data)
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(45000, 20) (45000, 3) (5000, 20) (5000, 3)


/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


### Neural Network

In [108]:
#import data
#path = "./not16nm_Delays_process_temp_pvdd_cqloadNEWLOAD.csv"
#data = import_data(path, ',')

#data = data.loc[1:, :]
#X = data.values[:, :-3]
#Y = data.values[:, -3:]

#X_train = np.asarray(X, dtype=np.float32)
#Y_train = np.asarray(Y, dtype=np.float32)
    
def NN(X_train,Y_train, multi="no"):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 42)
# Initialize the constructor
    model = Sequential()

    # Add an input layer and one hidden
    model.add(Dense(10, activation='tanh', input_shape=(X_train.shape[1],), kernel_initializer='normal'))

    model.add(Dense(5, activation='tanh', kernel_initializer='normal'))

    # Add an output layer 
    if multi == "yes":
        model.add(Dense(Y_train.shape[1], kernel_initializer='normal'))
    else:
        model.add(Dense(1, kernel_initializer='normal'))
        
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


    # #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)

    #train model
    model.fit(X_train, Y_train, validation_split=0.2, epochs=15, verbose=2, callbacks=[early_stopping_monitor])
    
    scores = model.evaluate(X_test, Y_test)
    
    return { "model" : model, "mse" : scores }

# results = model.fit(train_x, train_y, epochs=2, batch_size = 500, validation_data = (test_x, test_y))
#print("Test-Accuracy:", np.mean(results.history["val_acc"])))

#test
# test_y_predictions = model.predict(test_x)
# print(test_y_predictions)

# Accuracy
# print("test error: {}%".format(np.mean((test_y_predictions - test_y)**2) * 100))

# scores = model.evaluate(x=test_x, y=test_y, batch_size=None, verbose=2, sample_weight=None, steps=None)


# print('Test loss:', scores)

### Support Vector Regression for Mulitple Outputs

In [70]:
def SVMM(train_x,train_y, multi="no"):
    
    if multi == "yes":
        model1 = MultiOutputRegressor(SVR(kernel='rbf', C=1, gamma=0.1, epsilon=.1))
    else:
        model1 = SVR(kernel='rbf', C=1, gamma=0.1, epsilon=0.1)
    
    train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 42)
    
    predict_Y = model1.fit(train_x, train_y).predict(test_x)
    print(predict_Y.shape,test_y.shape)
#     score = np.square(np.sum(np.subtract(test_y, predict_Y)))*100/5000
    score_1 = mean_squared_error(test_y,predict_Y)
    #scores = model1.evaluate(x=test_x, y=test_y, batch_size=None, verbose=2, sample_weight=None, steps=None)
    #score1 = predict_Y.score(test_x, test_y)
#     print(score_1)
    return { "model" : model1, "mse" : score_1 }
#{ ( 1 / N ) * Σ [ (xi - x) * (yi - y) ] / (σx * σy ) }2
#x = np.subtract(predict_Y, np.mean(predict_Y))
#y = np.subtract(test_y, np.mean(test_y))
#v =np.multiply(np.std(predict_Y),np.std(test_y))
#u = np.multiply(v,5000)
#score2 = np.square(np.sum(np.multiply(x,y))/u)

#print(score2)


### Linear Regression 

In [71]:
def LR(train_x,train_y, multi="no"):
    
    if multi == "yes":
        model2 = MultiOutputRegressor(LinearRegression())
    else:
        model2 = LinearRegression()
    
    train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 42)

    model2.fit(train_x, train_y)
    predicty = model2.predict(test_x)
    #score2 = model1.score(test_x, test_y)
    score1 = mean_squared_error(predicty, test_y) 
    #score3 = r2_score(predicty, predicty) 
#     score2 = np.square(np.sum(np.subtract(test_y, predicty)))*100/5000
#     print(score1,score2)
    return { "model" : model2, "mse" : score1 } 


### Random Forest Regression

In [72]:
def RF(train_x,train_y,max_depth1=10, multi="no"):
    #max_depth1 = 2
    #max_depth2 = 10
    
    train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 42)

    model3 =  RandomForestRegressor(n_estimators=100, max_depth=max_depth1,
                                    random_state=2)
    #model4 =  RandomForestRegressor(n_estimators=100, max_depth=max_depth2,
     #                               random_state=2)

    model3.fit(train_x, train_y)
    #model4.fit(train_x, train_y)

    predict_y3= model3.predict(test_x)
    #predict_y4= model4.predict(test_x)

    #score3_1 = model3.score(test_x, test_y)
    score3_2 = mean_squared_error(predict_y3, test_y)
#     score3_3 = np.square(np.sum(np.subtract(test_y, predict_y3)))*100/5000

    #score4_1 = model4.score(test_x, test_y)
    #score4_2 = mean_squared_error(predict_y4, test_y)
    #score4_3 = np.square(np.sum(np.subtract(test_y, predict_y4)))*100/5000

#     print(score3_2,score3_3)
    return { "model" : model3, "mse" : score3_2 }
    #print(score4_2,score4_3)

### Decision Tree Regression

In [73]:
def DecisionTree(train_x,train_y,maxdepth, multi="no"):    
    
    train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 42)
    #maxdepth= 2
    model5 = DecisionTreeRegressor(max_depth=maxdepth)
    model5.fit(train_x, train_y)

    predict_y5 = model5.predict(test_x)

    #score5_1 = model3.score(test_x, test_y)
    score5_2 = mean_squared_error(predict_y5, test_y)
#     /score5_3 = np.square(np.sum(np.subtract(test_y, predict_y5)))*100/5000

#     print(score5_2, score5_3)
    return { "model" : model5, "mse" : score5_2 }

In [104]:
def Modelret(mod,train_x,train_y, multi="no"):
    if(mod=="SVM"):
        ret = SVMM(train_x,train_y, multi)
    elif(mod=="LR"):
        ret = LR(train_x,train_y, multi)
    elif(mod=="NN"):
        ret = NN(train_x,train_y, multi)
    elif(mod=="RF"):
        ret = RF(train_x,train_y,2, multi)
    elif(mod == "DT"):
        ret = DecisionTree(train_x,train_y,2, multi)
    return ret
        
        
        
        

In [129]:
def run(model, gate, X_test=None, Y_test=None):
    if gate == "AND3":
        data = import_data('./AND3/AND_3_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-6]
        Y = data[:, -6:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
#         print(X[:5, :])
#         print(Y[:5, :])
        mse = []
        
        ret_and3_del = Modelret(model, X_del, Y_del, "yes")
        mse.append(ret_and3_del["mse"])
    
        data = import_data('./AND3/000_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]
        
        ret_and3_leak000 = Modelret(model, X_000, Y_000)
        mse.append(ret_and3_leak000["mse"])
        
        data = import_data('./AND3/001_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_001 = data[:, :-1]
        Y_001 = data[:, -1]
        
        ret_and3_leak001 = Modelret(model, X_001, Y_001)
        mse.append(ret_and3_leak001["mse"])
        
        data = import_data('./AND3/010_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_010 = data[:, :-1]
        Y_010 = data[:, -1]
        
        ret_and3_leak010 = Modelret(model, X_010, Y_010)
        mse.append(ret_and3_leak010["mse"])
        
        data = import_data('./AND3/011_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_011 = data[:, :-1]
        Y_011 = data[:, -1]
        
        ret_and3_leak011 = Modelret(model, X_011, Y_011)
        mse.append(ret_and3_leak011["mse"])
        
        data = import_data('./AND3/100_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_100 = data[:, :-1]
        Y_100 = data[:, -1]
        
        ret_and3_leak100 = Modelret(model, X_100, Y_100)
        mse.append(ret_and3_leak100["mse"])
        
        data = import_data('./AND3/101_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_101 = data[:, :-1]
        Y_101 = data[:, -1]
        
        ret_and3_leak101 = Modelret(model, X_101, Y_101)
        mse.append(ret_and3_leak101["mse"])
        
        data = import_data('./AND3/110_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_110 = data[:, :-1]
        Y_110 = data[:, -1]
        
        ret_and3_leak110 = Modelret(model, X_110, Y_110)
        mse.append(ret_and3_leak110["mse"])
        
        data = import_data('./AND3/111_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_111 = data[:, :-1]
        Y_111 = data[:, -1]
        
        ret_and3_leak111 = Modelret(model, X_111, Y_111)
        mse.append(ret_and3_leak111["mse"])
        
        data = import_data('./AND3/000_AND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]
        
        
        mse = np.asarray(mse, dtype=np.float32)
        
        if X_test == None and model != "NN":
            print("MEAN SQUARED ERROR", np.sum(mse))
            return np.sum(mse)
            
    elif gate == "XOR2":
        data = import_data('./XOR2/XOR2_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-4]
        Y = data[:, -4:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
        #         print(X[:5, :])
        #         print(Y[:5, :])
        mse = []

        ret_XOR2_del = Modelret(model, X_del, Y_del, "yes")
        mse.append(ret_XOR2_del["mse"])

        data = import_data('./XOR2/00_XOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]

        ret_XOR2_leak000 = Modelret(model, X_000, Y_000)
        mse.append(ret_XOR2_leak000["mse"])

        data = import_data('./XOR2/01_XOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_001 = data[:, :-1]
        Y_001 = data[:, -1]

        ret_XOR2_leak001 = Modelret(model, X_001, Y_001)
        mse.append(ret_XOR2_leak001["mse"])

        data = import_data('./XOR2/10_XOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_010 = data[:, :-1]
        Y_010 = data[:, -1]

        ret_XOR2_leak010 = Modelret(model, X_010, Y_010)
        mse.append(ret_XOR2_leak010["mse"])

        data = import_data('./XOR2/11_XOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_011 = data[:, :-1]
        Y_011 = data[:, -1]

        ret_XOR2_leak011 = Modelret(model, X_011, Y_011)
        mse.append(ret_XOR2_leak011["mse"])

        if X_test == None and model != "NN":
            print("MEAN SQUARED ERROR", np.sum(mse))
            return np.sum(mse)
        
    elif gate == "NOR2":
        data = import_data('./NOR2/NOR2_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-4]
        Y = data[:, -4:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
        #         print(X[:5, :])
        #         print(Y[:5, :])
        mse = []

        ret_NOR2_del = Modelret(model, X_del, Y_del, "yes")
        mse.append(ret_NOR2_del["mse"])

        data = import_data('./NOR2/00_NOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]

        ret_NOR2_leak000 = Modelret(model, X_000, Y_000)
        mse.append(ret_NOR2_leak000["mse"])

        data = import_data('./NOR2/01_NOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_001 = data[:, :-1]
        Y_001 = data[:, -1]

        ret_NOR2_leak001 = Modelret(model, X_001, Y_001)
        mse.append(ret_NOR2_leak001["mse"])

        data = import_data('./NOR2/10_NOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_010 = data[:, :-1]
        Y_010 = data[:, -1]

        ret_NOR2_leak010 = Modelret(model, X_010, Y_010)
        mse.append(ret_NOR2_leak010["mse"])

        data = import_data('./NOR2/11_NOR2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_011 = data[:, :-1]
        Y_011 = data[:, -1]

        ret_NOR2_leak011 = Modelret(model, X_011, Y_011)
        mse.append(ret_NOR2_leak011["mse"])

        if X_test == None and model != "NN":
            print("MEAN SQUARED ERROR", np.sum(mse))
            return np.sum(mse)
    
    elif gate == "NAND2":
        data = import_data('./NAND2/NAND2_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-4]
        Y = data[:, -4:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
        #         print(X[:5, :])
        #         print(Y[:5, :])
        mse = []

        ret_NAND2_del = Modelret(model, X_del, Y_del, "yes")
        mse.append(ret_NAND2_del["mse"])

        data = import_data('./NAND2/00_NAND2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]

        ret_NAND2_leak000 = Modelret(model, X_000, Y_000)
        mse.append(ret_NAND2_leak000["mse"])

        data = import_data('./NAND2/01_NAND2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_001 = data[:, :-1]
        Y_001 = data[:, -1]

        ret_NAND2_leak001 = Modelret(model, X_001, Y_001)
        mse.append(ret_NAND2_leak001["mse"])

        data = import_data('./NAND2/10_NAND2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_010 = data[:, :-1]
        Y_010 = data[:, -1]

        ret_NAND2_leak010 = Modelret(model, X_010, Y_010)
        mse.append(ret_NAND2_leak010["mse"])

        data = import_data('./NAND2/11_NAND2.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_011 = data[:, :-1]
        Y_011 = data[:, -1]

        ret_NAND2_leak011 = Modelret(model, X_011, Y_011)
        mse.append(ret_NAND2_leak011["mse"])

        if X_test == None and model != "NN":
            print("MEAN SQUARED ERROR", np.sum(mse))
            return np.sum(mse)
        
    elif gate == "NAND3":
        data = import_data('./NAND3/NAND_3_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-6]
        Y = data[:, -6:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
#         print(X[:5, :])
#         print(Y[:5, :])
        mse = []
        
        ret_NAND3_del = Modelret(model, X_del, Y_del, "yes")
        mse.append(ret_NAND3_del["mse"])
    
        data = import_data('./NAND3/000_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]
        
        ret_NAND3_leak000 = Modelret(model, X_000, Y_000)
        mse.append(ret_NAND3_leak000["mse"])
        
        data = import_data('./NAND3/001_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_001 = data[:, :-1]
        Y_001 = data[:, -1]
        
        ret_NAND3_leak001 = Modelret(model, X_001, Y_001)
        mse.append(ret_NAND3_leak001["mse"])
        
        data = import_data('./NAND3/010_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_010 = data[:, :-1]
        Y_010 = data[:, -1]
        
        ret_NAND3_leak010 = Modelret(model, X_010, Y_010)
        mse.append(ret_NAND3_leak010["mse"])
        
        data = import_data('./NAND3/011_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_011 = data[:, :-1]
        Y_011 = data[:, -1]
        
        ret_NAND3_leak011 = Modelret(model, X_011, Y_011)
        mse.append(ret_NAND3_leak011["mse"])
        
        data = import_data('./NAND3/100_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_100 = data[:, :-1]
        Y_100 = data[:, -1]
        
        ret_NAND3_leak100 = Modelret(model, X_100, Y_100)
        mse.append(ret_NAND3_leak100["mse"])
        
        data = import_data('./NAND3/101_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_101 = data[:, :-1]
        Y_101 = data[:, -1]
        
        ret_NAND3_leak101 = Modelret(model, X_101, Y_101)
        mse.append(ret_NAND3_leak101["mse"])
        
        data = import_data('./NAND3/110_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_110 = data[:, :-1]
        Y_110 = data[:, -1]
        
        ret_NAND3_leak110 = Modelret(model, X_110, Y_110)
        mse.append(ret_NAND3_leak110["mse"])
        
        data = import_data('./NAND3/111_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_111 = data[:, :-1]
        Y_111 = data[:, -1]
        
        ret_NAND3_leak111 = Modelret(model, X_111, Y_111)
        mse.append(ret_NAND3_leak111["mse"])
        
        data = import_data('./NAND3/000_NAND3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]
        
        
        mse = np.asarray(mse, dtype=np.float32)
        
        if X_test == None and model != "NN":
            print("MEAN SQUARED ERROR", np.sum(mse))
            return np.sum(mse)
        
    elif gate == "NOR3":
        data = import_data('./NOR3/Nor_3_delay_16nm.csv', ',')
        data.head()
        data = data.values[1:, :]
        # print(data[:5, :])
        X = data[:, :-6]
        Y = data[:, -6:]
        X_del = np.asarray(X, dtype=np.float32)
        Y_del = np.asarray(Y, dtype=np.float32)
#         print(X[:5, :])
#         print(Y[:5, :])
        mse = []
        
        ret_NOR3_del = Modelret(model, X_del, Y_del, "yes")
        mse.append(ret_NOR3_del["mse"])
    
        data = import_data('./NOR3/000_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]
        
        ret_NOR3_leak000 = Modelret(model, X_000, Y_000)
        mse.append(ret_NOR3_leak000["mse"])
        
        data = import_data('./NOR3/001_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_001 = data[:, :-1]
        Y_001 = data[:, -1]
        
        ret_NOR3_leak001 = Modelret(model, X_001, Y_001)
        mse.append(ret_NOR3_leak001["mse"])
        
        data = import_data('./NOR3/010_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_010 = data[:, :-1]
        Y_010 = data[:, -1]
        
        ret_NOR3_leak010 = Modelret(model, X_010, Y_010)
        mse.append(ret_NOR3_leak010["mse"])
        
        data = import_data('./NOR3/011_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_011 = data[:, :-1]
        Y_011 = data[:, -1]
        
        ret_NOR3_leak011 = Modelret(model, X_011, Y_011)
        mse.append(ret_NOR3_leak011["mse"])
        
        data = import_data('./NOR3/100_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_100 = data[:, :-1]
        Y_100 = data[:, -1]
        
        ret_NOR3_leak100 = Modelret(model, X_100, Y_100)
        mse.append(ret_NOR3_leak100["mse"])
        
        data = import_data('./NOR3/101_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_101 = data[:, :-1]
        Y_101 = data[:, -1]
        
        ret_NOR3_leak101 = Modelret(model, X_101, Y_101)
        mse.append(ret_NOR3_leak101["mse"])
        
        data = import_data('./NOR3/110_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_110 = data[:, :-1]
        Y_110 = data[:, -1]
        
        ret_NOR3_leak110 = Modelret(model, X_110, Y_110)
        mse.append(ret_NOR3_leak110["mse"])
        
        data = import_data('./NOR3/111_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_111 = data[:, :-1]
        Y_111 = data[:, -1]
        
        ret_NOR3_leak111 = Modelret(model, X_111, Y_111)
        mse.append(ret_NOR3_leak111["mse"])
        
        data = import_data('./NOR3/000_NOR3.csv', ',')
        data = data.values[1:, :]
        data = np.asarray(data, dtype=np.float32)
        X_000 = data[:, :-1]
        Y_000 = data[:, -1]
        
        
        mse = np.asarray(mse, dtype=np.float32)
        
        if X_test == None and model != "NN":
            print("MEAN SQUARED ERROR", np.sum(mse))
            return np.sum(mse)
    

In [131]:
run("LR", "NAND3")

/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/

MEAN SQUARED ERROR 9.129159e-15


/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


9.129159e-15

In [128]:
gates = ["AND3", "XOR2"]
models = ["NN", "RF", "LR", "SVM", "DT"]


res = {}

for gate in gates:
    for model in models:
        if gate not in res:
            res[gate] = []
        res[gate].append(run(model, gate))

print(res)
    

/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Train on 36000 samples, validate on 9000 samples
Epoch 1/15
 - 3s - loss: 1.2460e-06 - acc: 0.2640 - val_loss: 1.8401e-22 - val_acc: 0.0000e+00
Epoch 2/15
 - 1s - loss: 4.6985e-21 - acc: 0.0098 - val_loss: 1.8401e-22 - val_acc: 0.0000e+00
Epoch 3/15
 - 1s - loss: 5.4361e-19 - acc: 0.1698 - val_loss: 1.2150e-17 - val_acc: 0.0000e+00
Epoch 4/15
 - 1s - loss: 2.8793e-10 - acc: 0.2844 - val_loss: 4.8126e-14 - val_acc: 0.0000e+00
5000/5000 [==============================] - 0s 28us/step


/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Train on 36000 samples, validate on 9000 samples
Epoch 1/15
 - 3s - loss: 2.7235e-06 - acc: 0.0000e+00 - val_loss: 2.5914e-19 - val_acc: 0.0000e+00
Epoch 2/15
 - 1s - loss: 2.3539e-19 - acc: 0.0000e+00 - val_loss: 2.5914e-19 - val_acc: 0.0000e+00
Epoch 3/15
 - 1s - loss: 2.3947e-19 - acc: 0.0000e+00 - val_loss: 2.2476e-19 - val_acc: 0.0000e+00
Epoch 4/15
 - 1s - loss: 4.7077e-19 - acc: 0.0000e+00 - val_loss: 2.2476e-19 - val_acc: 0.0000e+00
Epoch 5/15
 - 1s - loss: 7.3330e-19 - acc: 0.0000e+00 - val_loss: 2.7237e-18 - val_acc: 0.0000e+00
Epoch 6/15
 - 1s - loss: 4.1134e-09 - acc: 0.0000e+00 - val_loss: 1.3119e-16 - val_acc: 0.0000e+00
5000/5000 [==============================] - 0s 27us/step
Train on 36000 samples, validate on 9000 samples
Epoch 1/15
 - 3s - loss: 1.4241e-06 - acc: 0.0000e+00 - val_loss: 4.6642e-19 - val_acc: 0.0000e+00
Epoch 2/15
 - 1s - loss: 4.6123e-19 - acc: 0.0000e+00 - val_loss: 4.6642e-19 - val_acc: 0.0000e+00
Epoch 3/15
 - 1s - loss: 9.5487e-19 - acc: 0.0000e+0

/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns

...

/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(5000,) (5000,)
MEAN SQUARED ERROR 2.29631318423302e-12


/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pooja/.local/lib/python3.5/site-packages

MEAN SQUARED ERROR 6.670147326479663e-14
{'XOR2': [None, 6.670166761678655e-14, 6.6628676e-14, 2.29631318423302e-12, 6.670147326479663e-14], 'AND3': [None, 2.3354437e-16, 2.3275272e-16, 1.1969459e-14, 2.3355774e-16]}


/home/pooja/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
